In [1]:
import numpy as np
import random
from typing import Callable
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [78]:
class Neuron:
    def __init__(self, act_func:Callable[[float], float]=None):
        self.weights = None
        self.act_func = act_func

    def compile(self, weights_sz:int):
        self.weights = np.array([random.uniform(0, 10) for _ in range(weights_sz)]).reshape(weights_sz, 1)

    def forward(self, inp):
        z = inp @ self.weights
        return self.act_func(z)

In [79]:
class Layer:
    def sigmoid(self, x:float)-> float:
        return 1/(1+np.exp(-x))
    
    def __init__(self, neurons:int, act_fctn:str=None, name:str=None):
        self.name = name
        if act_fctn == "sigmoid":
            self.neurons = [Neuron(act_func=self.sigmoid) for i in range(neurons)]

    def __repr__(self):
        return self.name

    def compile(self, childs:int=None):
        for i in self.neurons:
            i.compile(childs)
        return len(self.neurons)

    def fit(self, inp):
        return np.squeeze(np.array([i.forward(inp) for i in self.neurons]), axis=2).T

In [80]:
class Tangrad:    
    def __init__(self, layer:list[Layer]=[], inp_features:int=None):
        self.layer = layer
        self.inp_features = inp_features
    
    def add_layer(self, layer:Layer):
        self.layer.append(layer)
    
    def compile(self):
        compiled = 0
        for i in range(len(self.layer)):
            if i == 0:
                compiled = self.layer[i].compile(self.inp_features)
            else:
                compiled = self.layer[i].compile(compiled)
        
    def fit(self, inp):
        for i in self.layer:
            inp = i.fit(inp)
        return inp

In [81]:
Ann = Tangrad(inp_features=4)

In [82]:
layer_0 = Layer(4, "sigmoid","layer 0")
layer_1 = Layer(20, "sigmoid","layer 1")
layer_2 = Layer(10, "sigmoid","layer 2")
layer_3 = Layer(1, "sigmoid","layer 3")
Ann.add_layer(layer_0)
Ann.add_layer(layer_1)
Ann.add_layer(layer_2)
Ann.add_layer(layer_3)

In [83]:
Ann.compile()

In [84]:
iris  = load_iris()

In [85]:
x = iris.data[:, :]
y = iris.target_names[iris.target] == "virginica"
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=85)

In [86]:
scaler = StandardScaler()
traindata = scaler.fit_transform(x_train)
testdata = scaler.fit_transform(x_test)

In [87]:
Ann.fit(traindata)

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],